python:3.9.13

tensorflow:2.18.0

torch:2.6.0

----

每局只取1个样本：慢  

缓慢降温增加探索性：效果不好。奖励只在最后一步才出现，探索性破坏奖励信号？  

更小卷积核：测试效果很差  

q-learning：测试效果很差  

sarsa：测试效果很差  

补充人工信息：效果提升

增加层数：加了两层，大部分时候表现优于aug，但是有交叉。

降低 learning rate: 从 1e-3 降低到 1e-4, 表现提升。

----

目前aug(base4基础上，输入数据增加了人工信息) 略优于 base4(补上了base2遗漏的batch normalization 和 dropout)

bigger(增加了两层) 略优于 aug

出牌信息里去掉带牌？可能意义不大

为什么训练过程中表现有起伏呢？降低 learning rate？增加每个轮次的局数？DQN？

如果DQN的话，要怎么做呢？DQN使用 experience buffer 是为了降低样本的相关性。在斗地主中，如果我一次收集很多局的样本，然后把样本打乱，喂给模型，这样就达到了降低样本相关性的目的吧。在这种情况下，policy network 本身就不是每一步都更新的。还有必要区分 policy network 和 target network 吗？

按照DQN的逻辑，我是不是可以把同一批样本反复随机分割成batch喂给模型？可以加快训练速度吗？

之前 q-learning 的效果太差了，完全没信心

----

trainer输出的loss意义有限。如果发牌高度偏向于地主或农民，那么预测就更准，loss就会更小。一轮只有8局，发牌的影响很大。

----

感觉 DouZero 所谓 Deep Monte-Carlo 跟我的逻辑是一样的。打牌，生成一整局游戏的样本，用最终的输赢作为预测目标，训练状态动作价值模型。多线程不熟悉代码不怎么看得懂，似乎一次生成50局游戏，然后 batch size 32

----
因为电脑是8核，之前都是每8局游戏更新模型。地主农民分三个模型之后，表现下降。可能是因为每个模型的样本数量只有之前的1/3。相应地改成每24局游戏更新一次，似乎表现提升，但是训练速度当然慢了很多。

一个问题，把收集到的样本打乱，32个样本一个batch，不满32个的也当成一个batch，是不是有问题？不满32个的应该怎么处理。假如某组只有1个样本，是不是不太好？

loss 修正: loss_factor = (len(xs_batch) / batch_size) ** 0.5  似无优势，可能训练过程略微平稳一些

----

怎么把douzero模型下载并跑起来……

----

tensorflow 模型 1696257 参数，运行速度 13ms，torch 模型 1457665 参数，运行速度 4ms。另外的我的模型需要的训练数据，组装速度也较慢，主要是组装额外数据(aug加入)消耗了额外的时间。

----

torch 比 tensorflow 快很多。发现用 torch 时，如果 batch size 为 1 会报错。所以 bn 是在一个 batch 内做 normalization？感觉不对劲。  
BatchNorm2D: batch 内所有样本同一个 channel 所有数据做 normalization, scaled(x[:, idx_channel, :, :])  
LayerNorm: batch 内每个样本单独做 normalization, scaled(x[idx_sample, :, :, :])  
算了，改用 LayerNorm 吧  
发现 LayerNorm 更平稳。为什么？ 直觉上似乎 batch normalization 更合理？因为有少数 batch size 不是固定的32？ batch size 太小？

----

似乎遇到内存泄露，跑了一晚上，报错停止了。  
使用 with mp.Pool() as pool, 内存稳定，但速度慢很多  
使用 del, 无效  
使用 del 和 gc.collect(), 无效，速度慢  
将就一下，仍旧使用 pool = mp.Poo() 以及每200轮关闭和重建创建 pool

----

公司个人办公电脑效率测试：trainer_torch_ln, 两分钟
cpu| rounds
----|----
1| 59
2| 79
3| 90
4| 94
5| 98
6| 102

公司linux服务器效率测试：trainer_torch_ln, 两分钟
cpu|rounds
----|----
1| 50
2| 58
3| 38
4| 13
6| 4

同样的代码，在我的公司电脑上运行的速度是在公司服务器/阿里云服务器上的20多倍。为什么？linux的问题？从资源占用来看，服务器上的cpu也是一直在工作的。  
找到原因了。模型计算时默认使用多核，各进程的模型互相抢CPU。不知道为什么这个问题在个人PC上没那么严重，windows和linux的差异？  
torch.set_num_threads 设置使用模型使用的线程数。在linux服务器上设置12进程，子进程 torch.set_num_threads(1) 之后，两分钟跑了80轮  
本地PC采用8进程和torch.set_num_threads(1)之后，运行速度似有些微提升。

-----
使用 python multiprocessing 多进程遇到的问题：  
内存占用不断增加：使用 with mp.Pool() as pool 可以避免这个问题，但是会降低程序运行速度。改成在循环中每隔若干轮关闭并重建pool。  
使用 Process(target, args)，不一定使用多核。不知道如何指定使用几个进程。即使使用多核，使用队列传递数据，程序运行速度相比 pool.map 也不见优势。  
使用多进程，运行速度没有线性增加。  
使用6核，设定每个 selfplay 进行一局游戏，执行24次，或者设定每个selfplay 进行4局游戏，执行6次，运行效率没区别。但把总执行次数改成48局，前一种会稍慢。说明复制参数或收集返回结果确实有计算成本，只是在本电脑本问题上不是很显著。  

----
Dropout: 同样的输入，在 train 模式下，输出会不一样。但是如果输出的方差很大，这会不会是个问题？  
刚才是用随机生成的数据测试。重新用实际的牌局数据测试，方差不大。

----
不合理出牌：  
bot_torch_ln: 316300  

```
0.99337137 6 A A A
0.9860233 6 6 A A A
0.985545 3 A A A
0.9849531 3
0.9831738 A A A
21 :  地主 : 6 A A A | 3 6 大王
22 : 农民1 :  | 4 5 7 7 J Q Q K K 2
23 : 农民2 :  | 3 3 4 4 4 5 5 10 2 2
```

----

每100轮(24局1轮)存1次模型，与douzero比较，地主农民各1000局，统计45万轮之后各模型的胜利局数。分开看做地主和做农民时的胜利局数，统计的方差和基于胜率估算的方差相差不大。但是把做地主和做农民胜利的局数加起来看，统计的方差和基于胜率估算的方差相差很大。地主胜率和农民胜率相关系数 -0.6。这种情况给人什么感觉呢？好像模型实力已经不怎么提升了，只是在左右互搏的过程中限入了某种循环状态。类似 地主策略 A > 农民策略 A > 地主策略B > 农民策略 B > 地主策略 A

使用最近200轮的模型池生成样本能避免这种情况吗？目前从30万轮起，跑了7万轮，地主胜率和农民胜率相关系数同样 -0.6。继续跑看看。

上面的猜测好像不对。测试的时候，每组发牌打两局，一局当地主，一局当农民。如果测试某一轮的模型时，发的牌整体偏向地主，地主胜率就高，农民胜率就低

----

从前200个模型中随机抽取模型对战生成样本，避免可能的策略过拟合：看不出来效果。

----

根据模型打牌，生成一个局面信息的序列。然后在序列的每一步，选择其他（最多4手）评分高的出牌选项，接下来再按模型打牌，生成序列。  
如此得到一个局面信息的树，每一层有若干个子结点，对应当前局面下的不同出牌，但只有一个子结点（最高胜率的出牌）有分支，其他的都是一条线直到叶结点。  
即先按模型生成主干，然后在主干上的每一步分出分支，每个分支不再有新的分支，而是按模型一路走到底。  
希望具有的优点：1, 一手牌提供更多的训练数据。2, 同一个局面下的不同出牌的奖励对比能提供更有效的训练信号。  
还在训练中……  
试验几种设置：一轮6局，batch size=32 | 一轮24局，batch size=32 | 一轮24局，batch size=128  
一轮24局，batch size=128 较好  
训练似乎较慢。每局游戏生成的样本数量大大增加（50多倍），每一轮的耗时也大大增加。虽然样本多，但对应的初始发牌少，训练速度慢可能与此有关。继续训练希望表现能更好。

表现似乎有所提升，对douzero胜率大概提升了0.5%

----
不作弊的模型  
128局一轮优于24局一轮  
增加自己手牌的顺子、连对、飞机信息，表现较大提升  
使用 lstm_out 和 h_n 作为下一步的输入，表现无差异

# 加载包

In [1]:
import numpy as np
import keras as k
import tensorflow as tf
import torch
from torch import nn
import rules
import arena as arn
import bot_torch_ln
import bot_aug
CARDS = rules.CARDS
ARENA = arn.ARENA
bot_rd = arn.bot_rd
bot_me = arn.bot_me


In [2]:
import bot_douzero
import importlib
importlib.reload(bot_douzero)

<module 'bot_douzero' from 'd:\\py\\ddz_2025_2\\bot_douzero.py'>

In [3]:
def ft(N, bot1_f, bot2_f):
    s1, s2 = 0, 0
    for i in range(N):
        cards = rules.CARDS.copy()
        np.random.shuffle(cards)
        
        arena = ARENA(verbos=0, cards=cards.copy())
        arena.registerbot([bot1_f(), bot2_f(), bot2_f()])
        arena.wholegame()
        s1 += arena.winner == 0

        arena = ARENA(verbos=0, cards=cards.copy())
        arena.registerbot([bot2_f(), bot1_f(), bot1_f()])
        arena.wholegame()
        s2 += arena.winner == 0

    return s1, s2

# models & bots

In [4]:
import randbot_three

In [5]:
#451500: 2089, 1542 | 5547/10000
#499000: 2078, 1594 | 5484/10000
#400900: 2135, 1656 | 5479/10000
#473700: 2129, 1610 | 5519/10000
#520500: 2157, 1618 | 5539/10000
#672200: 2135, 1533 | 5602/10000
#701000: 2184, 1556 | 5628/10000
#745700：2112, 1521 | 5591/10000
cp = torch.load("model_torch_ln/cp701000.pt")
models = []
for pos in range(3):
    model = bot_torch_ln.Model()
    model.eval()
    model.load_state_dict(cp["models_state_dict"][pos])
    models.append(model)

In [6]:
# 7489: 2143, 1644 | 5499/10000
# 7712: 2126, 1682 | 5440/10000
#12762: 2127, 1509 | 5618/10000
#12782: 2106, 1478 | 5628/10000
#13339: 2163, 1494 | 5669/10000
#14315: 2188, 1509 | 5679/10000  vs model_torch_ln/cp701000: 1677, 1630 / 5000
#14384: 2026, 1439 | 5587/10000
#14427: 2036, 1507 | 5529/10000
#14671: 2088, 1481 | 5607/10000
#15783: 2093, 1436 | 5657/10000
cp_tree = torch.load("model_tree2/cp14315.pt")
models_tree = []
for pos in range(3):
    model = bot_torch_ln.Model()
    model.eval()
    model.load_state_dict(cp_tree["models_state_dict"][pos])
    models_tree.append(model)

In [7]:
def bot_torch_ln_f(verbos=0):
    return bot_torch_ln.BOT(models, verbos=verbos)

def bot_tree_f(verbos=0):
    return bot_torch_ln.BOT(models_tree, verbos=verbos)

def bot_rand_f(verbos=0, sample=10):
    return randbot_three.BOT([bot_torch_ln_f(0), bot_torch_ln_f(0), bot_torch_ln_f(0)], verbos=verbos, sample=sample)

model_aug = k.models.load_model("e:/ddz_2025_2_model/model_aug/m488750.keras")
def bot_aug_f(verbos=0):
    return bot_aug.BOT(model_aug, verbos)

def bot_rd_f():
    return bot_rd

def bot_douzero_f(verbos=0):
    return bot_douzero.BOT(verbos)

In [13]:
N = 5000
# ft(N, bot_torch_ln_f, bot_douzero_f)  ##623
# ft(N, bot_douzero_f, bot_douzero_f) ##66
ft(N, bot_tree_f, bot_torch_ln_f)
# ft(N, bot_tree_f, bot_douzero_f)

(1677, 1630)

# 示例

In [8]:
arena = arn.ARENA(1, True)
# arena.registerbot([bot_torch_ln_f(1), bot_douzero_f(1), bot_douzero_f(1)])
arena.registerbot([bot_tree_f(1), bot_tree_f(1), bot_tree_f(1)])
arena.wholegame()

0.037720356 3 3
0.03566752 4 4
0.019491475 3
0.018694827 8
0.01810125 6
 0 :  地主 : 3 3 | 4 4 5 6 6 7 8 8 9 10 10 Q Q K A 2 小王 大王
0.9358061 7 7
0.9322855 5 5
0.93197274 
0.90656066 J J
0.89731216 K K
 1 : 农民1 : 7 7 | 3 3 5 5 8 9 10 J J Q K K A 2 2
0.9014698 
0.8112476 A A
0.79149675 9 9
0.7345604 J J
 2 : 农民2 :  | 4 4 5 6 6 7 8 9 9 10 J J Q K A A 2
0.01725153 8 8
0.016592976 10 10
0.014270077 Q Q
0.0063977353 
0.00023042299 小王 大王
 3 :  地主 : 8 8 | 4 4 5 6 6 7 9 10 10 Q Q K A 2 小王 大王
0.98421067 
0.9826357 K K
0.9805629 J J
0.9623565 2 2
 4 : 农民1 :  | 3 3 5 5 8 9 10 J J Q K K A 2 2
0.9318038 9 9
0.9243262 
0.92123544 J J
0.9170981 A A
 5 : 农民2 : 9 9 | 4 4 5 6 6 7 8 10 J J Q K A A 2
0.040947463 10 10
0.03745124 Q Q
0.011923501 
0.00020095741 小王 大王
 6 :  地主 : 10 10 | 4 4 5 6 6 7 9 Q Q K A 2 小王 大王
0.953721 K K
0.9505284 
0.9236219 J J
0.90811497 2 2
 7 : 农民1 : K K | 3 3 5 5 8 9 10 J J Q A 2 2
0.8804683 
0.7927585 A A
 8 : 农民2 :  | 4 4 5 6 6 7 8 10 J J Q K A A 2
0.010611471 
0.00022327603 小王 大

# 同样发牌交换地主农民

In [28]:
while True:
    cards = CARDS.copy()

    np.random.shuffle(cards)
    arena = ARENA(cards=cards.copy())
    arena.registerbot([bot_tree_f(), bot_douzero_f(), bot_douzero_f()])
    arena.wholegame()
    w1 = arena.winner
    
    arena = ARENA(cards=cards.copy())
    arena.registerbot([bot_douzero_f(), bot_tree_f(), bot_tree_f()])
    arena.wholegame()
    w2 = arena.winner
    
    if (w1==0) != (w2==0):
        arena = ARENA(cards=cards)
        break

In [29]:
arena = ARENA(1, cards=cards.copy())
arena.registerbot([bot_tree_f(1), bot_douzero_f(1), bot_douzero_f(1)])
arena.wholegame()

0.044815898 3 4 5 6 7
0.012974928 3 4 5 6 7 8 9
0.012666023 8 8 8 2
0.011489981 8 8 8 9
0.010719384 K K
 0 :  地主 : 3 4 5 6 7 | 8 8 8 9 10 10 J J Q Q Q K K A 2
0.4301774 10 J Q K A
0.31327137 9 10 J Q K
0.30184177 
 1 : 农民1 : 10 J Q K A | 3 4 5 5 7 7 7 9 9 2 2 大王
 2 : 农民2 :  | 3 3 4 4 5 6 6 6 8 9 10 J K A A 2 小王
0.025917644 
 3 :  地主 :  | 8 8 8 9 10 10 J J Q Q Q K K A 2
0.47141758 9 9
0.46067223 5 5
0.4533853 3 7 7 7
0.4356561 3
0.42090124 5 5 7 7 7
 4 : 农民1 : 9 9 | 3 4 5 5 7 7 7 2 2 大王
0.6435713 
0.2126899 A A
 5 : 农民2 :  | 3 3 4 4 5 6 6 6 8 9 10 J K A A 2 小王
0.030402439 10 10
0.014142957 J J
0.011686614 
0.008780964 K K
0.0015863057 Q Q
 6 :  地主 : 10 10 | 8 8 8 9 J J Q Q Q K K A 2
0.4556514 
0.3229889 2 2
 7 : 农民1 :  | 3 4 5 5 7 7 7 2 2 大王
0.34693265 A A
0.32949412 
 8 : 农民2 : A A | 3 3 4 4 5 6 6 6 8 9 10 J K 2 小王
0.06705099 
 9 :  地主 :  | 8 8 8 9 J J Q Q Q K K A 2
0.36528406 
0.14076768 2 2
10 : 农民1 :  | 3 4 5 5 7 7 7 2 2 大王
0.51167566 3 3
0.5108242 4 4
0.50972074 6 6
0.40327597 4 4 

In [30]:
arena = ARENA(1, cards=cards.copy())
arena.registerbot([bot_douzero_f(1), bot_tree_f(1), bot_tree_f(1)])
arena.wholegame()

-0.49775583 3 4 5 6 7
-0.6626981 8 8
-0.6694203 3 4 5 6 7 8
-0.74220794 8
-0.79250985 3
 0 :  地主 : 3 4 5 6 7 | 8 8 8 9 10 10 J J Q Q Q K K A 2
0.982307 9 10 J Q K
0.9822033 10 J Q K A
0.92688024 
 1 : 农民1 : 9 10 J Q K | 3 4 5 5 7 7 7 9 A 2 2 大王
0.9884756 
 2 : 农民2 :  | 3 3 4 4 5 6 6 6 8 9 10 J K A A 2 小王
-0.30485788 10 J Q K A
-0.838125 
 3 :  地主 : 10 J Q K A | 8 8 8 9 10 J Q Q K 2
0.89351285 
 4 : 农民1 :  | 3 4 5 5 7 7 7 9 A 2 2 大王
0.87796545 
 5 : 农民2 :  | 3 3 4 4 5 6 6 6 8 9 10 J K A A 2 小王
0.11485767 9 10 J Q K
-0.088388056 8 8 8 Q
-0.3348835 8 8 8 2
-0.5829635 Q
-0.6525965 8 8 8
 6 :  地主 : 9 10 J Q K | 8 8 8 Q 2
0.12513119 
 7 : 农民1 :  | 3 4 5 5 7 7 7 9 A 2 2 大王
0.13149017 
 8 : 农民2 :  | 3 3 4 4 5 6 6 6 8 9 10 J K A A 2 小王
0.1280784 8 8 8 Q
-0.12754178 8 8 8 2
-0.63166255 8 8 8
-0.79782104 2
-0.802111 Q
 9 :  地主 : 8 8 8 Q | 2
0.052092493 
10 : 农民1 :  | 3 4 5 5 7 7 7 9 A 2 2 大王
3.373623e-05 
11 : 农民2 :  | 3 3 4 4 5 6 6 6 8 9 10 J K A A 2 小王
12 :  地主 : 2 | 


# 测试不同轮次的地主和农民

In [10]:
models = []
cp = torch.load("model_torch_ln/cp520500.pt")
model = bot_torch_ln.Model()
model.eval()
model.load_state_dict(cp["models_state_dict"][0])
models.append(model)

cp = torch.load("model_torch_ln/cp451500.pt")

model = bot_torch_ln.Model()
model.eval()
model.load_state_dict(cp["models_state_dict"][1])
models.append(model)

model = bot_torch_ln.Model()
model.eval()
model.load_state_dict(cp["models_state_dict"][2])
models.append(model)

def bot_torch_ln_f(verbos=0):
    return bot_torch_ln.BOT(models, verbos=verbos)

In [11]:
N = 5000
# ft(N, bot_three_f, bot_three_f)  ##623
# ft(N, bot_douzero_f, bot_douzero_f) ##66
ft(N, bot_torch_ln_f, bot_douzero_f)

(2166, 1620)

# 根据发牌构建arena

In [6]:
import re

def repl(matched):
    d = {"11": "J", "12": "Q", "13": "K", "14": "A", "15": "2", "16": "小王", "17": "大王"}
    return d[matched[0]]

def arenabyscratch(l):
    cards = [rules.str2vec(re.sub(r'1[1-7]', repl, v)) for v in re.split(",", l)]
        
    arena=object.__new__(arn.ARENA)
    arena.verbos = 1
    arena.init = np.array(cards, int)
    arena.remain = arena.init.copy()
    arena.lastplay = np.zeros((3, 15), int)
    arena.pos = 0
    arena.b1 = 2
    arena.b2 = 1
    arena.round = 0
    arena.bot = []
    arena.gameover = False
    arena.winner = None
    arena.getdata()
    arena.RECORD = True
    arena.records = []
    if arena.RECORD:
        arena.records.append(arena.copy())
    return arena

arena = arenabyscratch("3 4 4 4 4 5 5 7 8 9 9 9 10 11 12 13 13 14 14 17, 3 5 5 6 6 6 7 7 7 8 9 10 11 12 13 13 15, 3 3 6 8 8 10 10 11 11 12 12 14 14 15 15 15 16")

# 草稿

In [11]:
len(arena.records)

36

In [13]:
len(arena.history)

35

In [17]:
arena.records[-2].remain

array([[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 2, 1, 2, 1, 1, 0, 1, 2, 1, 1, 2, 1, 0, 0]])

In [18]:
arena.history[-1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])